In [ ]:
!python --version

: 

In [ ]:
%%capture

# The environment we will be working with
%pip install CityLearn==2.1.2

# For participant interactions (buttons)
%pip install ipywidgets

# To generate static figures
%pip install matplotlib
%pip install seaborn

# Provide standard RL algorithms
%pip install stable-baselines3

# Enable gym compatibility with later stable-baselines3 versions
%pip install shimmy

# Results submission
%pip install requests
%pip install beautifulsoup4

In [ ]:
import sys, subprocess
print("Python:", sys.executable)
print("Pip:   ", subprocess.run(["which","pip"], capture_output=True, text=True).stdout)

# System operations
import os

# Type hinting
from typing import Any, List, Mapping, Tuple, Union

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import parallel_coordinates

import import_ipynb

# Data manipulation
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
import torch as th

# CityLearn
from citylearn.data import DataSet
from citylearn.reward_function import RewardFunction, SolarPenaltyReward

# Baseline RL algorithms
from stable_baselines3 import SAC
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecMonitor, VecNormalize
from stable_baselines3.common.callbacks import EvalCallback, CallbackList
from stable_baselines3.common.monitor import Monitor


# set all plotted figures without margins
plt.rcParams['axes.xmargin'] = 0
plt.rcParams['axes.ymargin'] = 0
%matplotlib inline

In [ ]:
# Dataset
DATASET_NAME = 'citylearn_challenge_2023_phase_3_1'
schema = DataSet().get_schema(DATASET_NAME)
print(schema['root_directory'])

# Building
#root_directory = schema['root_directory']
root_directory = 'Bachelorthesis_DQN_Agent/data/datasets/citylearn_challenge_2023_phase_3_1'
building_name = 'Building_1'
# Weather data
filename = schema['buildings'][building_name]['weather']
filepath = os.path.join(root_directory, filename)
weather_data = pd.read_csv(filepath)
# Pricing data (simple)
filename = schema['buildings'][building_name]['pricing']
filepath = os.path.join(root_directory, filename)
pricing_data = pd.read_csv(filepath)
# Carbon Intensity data
filename = schema['buildings'][building_name]['carbon_intensity']
filepath = os.path.join(root_directory, filename)
carbon_intensity_data = pd.read_csv(filepath)
# building data
filename = schema['buildings'][building_name]['energy_simulation']
filepath = os.path.join(root_directory, filename)
building_data = pd.read_csv(filepath)

# Display building data
# display(building_data.head())
# display(building_data.describe(include='all'))

bld = building_data.copy()
wth = weather_data.copy()
prc = pricing_data.copy()
car = carbon_intensity_data.copy()

print(building_data.columns.tolist())

In [ ]:
observations = [
    # building_df
    'month', 'hour', 'day_type', 'daylight_savings_status',
    'indoor_dry_bulb_temperature',
    'average_unmet_cooling_setpoint_difference',
    'indoor_relative_humidity',
    'non_shiftable_load', 'dhw_demand',
    'cooling_demand', 'heating_demand',
    'solar_generation', 'occupant_count',
    'indoor_dry_bulb_temperature_cooling_set_point',
    'indoor_dry_bulb_temperature_heating_set_point', 'hvac_mode',
    # weather_df
    'outdoor_dry_bulb_temperature',
    'outdoor_relative_humidity',
    'diffuse_solar_irradiance',
    'direct_solar_irradiance',
    'outdoor_dry_bulb_temperature_predicted_1',
    'outdoor_dry_bulb_temperature_predicted_2',
    'outdoor_dry_bulb_temperature_predicted_3',
    'outdoor_relative_humidity_predicted_1',
    'outdoor_relative_humidity_predicted_2',
    'outdoor_relative_humidity_predicted_3',
    'diffuse_solar_irradiance_predicted_1',
    'diffuse_solar_irradiance_predicted_2',
    'diffuse_solar_irradiance_predicted_3',
    'direct_solar_irradiance_predicted_1',
    'direct_solar_irradiance_predicted_2',
    'direct_solar_irradiance_predicted_3',
    # carbon_df 
    'carbon_intensity',
    # pricing_df
    'electricity_pricing',
    'electricity_pricing_predicted_1',
    'electricity_pricing_predicted_2',
    'electricity_pricing_predicted_3'
]
bld_cols = [
            'month', 'hour', 'day_type', 'daylight_savings_status',
            'indoor_dry_bulb_temperature',
            'average_unmet_cooling_setpoint_difference',
            'indoor_relative_humidity', 'non_shiftable_load',
            'dhw_demand', 'cooling_demand', 'heating_demand',
            'solar_generation', 'occupant_count',
            'indoor_dry_bulb_temperature_cooling_set_point',
            'indoor_dry_bulb_temperature_heating_set_point',
            'hvac_mode'
]
wth_cols = [
    'outdoor_dry_bulb_temperature',
    'outdoor_relative_humidity',
    'diffuse_solar_irradiance',
    'direct_solar_irradiance'
]

### Deep Q-Learning Agent
#### 1. Reward Function


In [ ]:
class CustomReward(RewardFunction):
    def __init__(self, capacity: float):
        self.capacity = capacity
        self.prev_net_load = None

    def __call__(self, obs: Mapping[str, float], action_frac: float) -> float:
        # 1) Net load & cost
        net_load   = obs['non_shiftable_load'] - action_frac*self.capacity # max: 11.25 kW
        price = obs['electricity_pricing'] # 0.06605 or 0.03025
        cost  = max(0.0, net_load) * price # deviation range: 0-11.25*0.06605 = ~0-0.75$
        #norm_cost = cost / 0.75 # normalize to [0,1]

        # return -(w_cost*cost + w_pen*comfort_penalty + w_emis*emis + w_ramp*ramp)        
        return -cost

In [ ]:
class TrainLoggerCallback(BaseCallback):
    """Logging State, Action, Reward per step and Loss per update phase."""
    def __init__(self, verbose=0):
        super().__init__(verbose)
        # Will collect a dict per env-step
        self.rows = []
        # Loss values and their timesteps (global)
        self.losses = []
        self.loss_timesteps = []
        # Completed episode returns (global list)
        self.episode_rewards = []
        # Placeholders for per-env tracking
        self._current_ep_rewards = []         # sum of rewards in current episode per env
        self._current_ep_counts = []          # episode index per env
        self._current_step_in_episode = []    # step counter (0..T-1) per env

        # DataFrames to populate at end
        self.df = pd.DataFrame()
        self.ep_df = pd.DataFrame()

    def _on_training_start(self) -> None:
        try:
            n_envs = self.training_env.num_envs
        except AttributeError:
            n_envs = 1
        # initialize counters per sub-env
        self._current_ep_rewards = [0.0] * n_envs
        self._current_ep_counts = [1] * n_envs
        self._current_step_in_episode = [0] * n_envs
        super()._on_training_start()

    def _on_step(self) -> bool:
        obs_vec = self.locals.get("new_obs")
        acts    = self.locals.get("actions")
        rews    = self.locals.get("rewards")
        dones   = self.locals.get("dones")
        step    = int(self.num_timesteps)

        # log loss if present
        loss_val = self.logger.name_to_value.get("train/loss")
        if loss_val is not None:
            self.losses.append(float(loss_val))
            self.loss_timesteps.append(step)

        # iterate each sub-env
        for idx, (obs, act, rew, done) in enumerate(zip(obs_vec, acts, rews, dones)):
            # flatten observation
            flat = obs.flatten().tolist()
            # build row with metadata
            row = {f"x{i}": flat[i] for i in range(len(flat))}
            row.update({
                "env_id": idx,
                "episode": self._current_ep_counts[idx],
                "step_in_ep": self._current_step_in_episode[idx],
                "action": int(act),
                "reward": float(rew),
                "global_step": step
            })
            self.rows.append(row)

            # accumulate per-episode reward
            self._current_ep_rewards[idx] += float(rew)
            # increment step in episode
            self._current_step_in_episode[idx] += 1

            # if end of episode for this env
            if done:
                # log reward and finalize episode
                print(f"Env {idx} Episode {self._current_ep_counts[idx]} done at global step {step}, total reward: {self._current_ep_rewards[idx]:.3f}")
                self.episode_rewards.append(self._current_ep_rewards[idx])
                # reset for next episode
                self._current_ep_rewards[idx] = 0.0
                self._current_ep_counts[idx] += 1
                self._current_step_in_episode[idx] = 0

        return True

    def _on_training_end(self) -> None:
        # build full-step DataFrame
        self.df = pd.DataFrame(self.rows)
        # build episodes summary DataFrame
        self.ep_df = pd.DataFrame({
            "episode_global": range(1, len(self.episode_rewards) + 1),
            "return": self.episode_rewards
        })
        super()._on_training_end()

In [ ]:
# Minimal DQN on standard CityLearnEnv (single building, single action)
import numpy as np
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor

from citylearn.data import DataSet
from citylearn.citylearn import CityLearnEnv

# --- keep only electrical storage so we have 1D continuous action to discretize ---
def keep_only_electrical_storage(schema: dict) -> dict:
    if 'actions' in schema:
        for a in list(schema['actions'].keys()):
            schema['actions'][a]['active'] = (a == 'electrical_storage')
    if 'buildings' in schema:
        for b in schema['buildings']:
            if 'actions' in b:
                for a in list(b['actions'].keys()):
                    b['actions'][a]['active'] = (a == 'electrical_storage')
    return schema

# --- wrapper: single-agent SB3 env on top of CityLearnEnv + action discretization ---
class CityLearnDQNWrapper(gym.Env):
    """Wrap standard CityLearnEnv for SB3 DQN:
       - single central agent, we control building 0
       - discretize the [-1, 1] storage action into n_bins
    """
    metadata = {"render_modes": []}

    def __init__(self, cl_env: CityLearnEnv, n_bins: int = 5):
        super().__init__()
        self.cl = cl_env
        self.n_bins = int(n_bins)
        assert self.n_bins >= 2, "n_bins must be >= 2"

        # Build discrete action space
        self.action_space = spaces.Discrete(self.n_bins)

        # Infer observation shape (use building 0 obs)
        self.cl.reset()
        obs0 = np.array(self.cl.observations[0], dtype=np.float32)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=obs0.shape, dtype=np.float32
        )

        # Precompute action broadcast size (how many controls for bldg 0)
        self.N = self.cl.action_space[0].shape[0]  # should be 1 if only electrical_storage is active

    def _map_discrete_to_frac(self, a: int) -> float:
        # map {0..n_bins-1} -> [-1, 1]
        if self.n_bins == 1:
            return 0.0
        return -1.0 + 2.0 * (a / (self.n_bins - 1))

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        self.cl.reset(seed=seed)
        obs = np.array(self.cl.observations[0], dtype=np.float32)
        return obs, {}

    def step(self, a: int):
        # map discrete to continuous fraction
        frac = float(np.clip(self._map_discrete_to_frac(int(a)), -1.0, 1.0))
        # build CityLearn action format: list per building -> flat list for building 0
        actions = [[frac] * self.N]  # control building 0 only
        obs_all, rewards, terminated, truncated, _info = self.cl.step(actions)

        obs = np.array(obs_all[0], dtype=np.float32)
        reward = float(rewards[0])
        term = bool(terminated)
        trunc = bool(truncated)
        return obs, reward, term, trunc, {}

# ----------------- build env -----------------
dataset = DataSet()
schema = dataset.get_schema('citylearn_challenge_2023_phase_3_1')
schema = keep_only_electrical_storage(schema)
cl_env = CityLearnEnv(schema, central_agent=True)

env = Monitor(CityLearnDQNWrapper(cl_env, n_bins=5))

# ----------------- train DQN (super simple) -----------------
model = DQN(
    "MlpPolicy",
    env,
    learning_rate=3e-4,
    buffer_size=50_000,
    batch_size=256,
    learning_starts=2_000,
    train_freq=256,
    target_update_interval=1_000,
    gamma=0.98,
    exploration_fraction=0.3,
    exploration_initial_eps=1.0,
    exploration_final_eps=0.05,
    verbose=1,
    policy_kwargs=dict(net_arch=[256, 256], activation_fn=None),
)

# pick a small budget to smoke-test; increase later
model.learn(total_timesteps=100_000)

# ----------------- quick evaluation (one episode) -----------------
obs, _ = env.reset()
done = False
ret = 0.0
while not done:
    action = model.predict(obs, deterministic=True)[0]
    obs, r, term, trunc, _ = env.step(action)
    ret += r
    done = term or trunc
print(f"Episode return: {ret:.3f}")


In [ ]:
# Training reward
window = 100
smooth = train_callback.ep_df["return"].rolling(window, min_periods=1).mean()
#train_df = train_callback.df  # your TrainLoggerCallback should have produced this
plt.figure(figsize=(6, 3))
plt.plot(train_callback.ep_df["episode_global"], smooth, label=f"{window}-Episode MA", color="C0")
#plt.plot(train_df["step"], train_df["reward"], color="C0")
plt.title("Moving Average of the Reward")
plt.xlabel("Episode")
plt.ylabel("Reward (moving average)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
n_episodes = len(train_callback.ep_df)
print(f"Total episodes: {n_episodes}")
"""
# Training loss (if available)
if hasattr(train_callback, "losses") and len(train_callback.losses) > 0:
    plt.figure(figsize=(6, 3))
    plt.plot(train_callback.loss_timesteps,
             train_callback.losses,
             marker='.', linestyle='-',
             alpha=0.7, color="C1")
    plt.title("Train Loss over time")
    plt.xlabel("Timesteps")
    plt.ylabel("Loss")
    plt.grid(True)
    plt.tight_layout()
    plt.show()
"""
train_df = train_callback.df  

# Actions (raw + moving average)
plt.figure(figsize=(6, 3))
# raw scatter
plt.scatter(
    train_df["global_step"],
    train_df["action"],
    c="lightgray", s=5, alpha=0.4,
    label="raw actions"
)
# moving average
window = 500
train_df["action_ma"] = train_df["action"].rolling(window, min_periods=1).mean()
plt.plot(
    train_df["global_step"],
    train_df["action_ma"],
    color="C2", lw=2,
    label=f"{window}-step MA"
)
plt.title("Actions over time (raw + moving average)")
plt.xlabel("Timesteps")
plt.ylabel("Discrete Action")
plt.legend(loc="upper right")
plt.tight_layout()
plt.show()

# Evaluation results
all_eval = s.evaluations_results        # list of lists
steps    = eval_callback.evaluations_timesteps      # list of ints

# Compute per‐evaluation mean/std
mean_eval = [float(np.mean(r)) for r in all_eval]
std_eval  = [float(np.std(r))  for r in all_eval]

plt.figure(figsize=(6, 3))
plt.errorbar(
    steps, mean_eval,
    yerr=std_eval,
    fmt='-o', capsize=3, color="C3",
    label="Eval mean ±1σ"
)
plt.title("Mean Evaluation Reward over time")
plt.xlabel("Timesteps")
plt.ylabel("Mean Reward")
plt.grid(True)
plt.tight_layout()
plt.show()